<a href="https://colab.research.google.com/github/kevin6449/ironman2024_genai/blob/main/gen_ai_day22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 瞭解及計算符記
Gemini 和其他生成式 AI 模型會精細處理輸入和輸出內容 符記。



### 關於權杖
符記可以是單一字元 (例如 z) 或完整的字詞 (例如 cat)。詳細字詞 可分為多個符記模型使用的所有符記集 將文字分割成符記的過程 符記化。

Gemini 模型的符記約為 4 個字元。 100 個符記約等於 60 到 80 個英文單字。

啟用帳單功能後，對 Gemini API 的呼叫費用為 有些部分取決於輸入和輸出符記的數量，因此知道 計算符記就能派上用場

In [ ]:
import google.generativeai as genai

from google.colab import userdata

API_KEY=userdata.get('GOOGLE_API_KEY')

#genai.configure(api_key="YOUR_API_KEY")

# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

## 上下文視窗

透過 Gemini API 提供的模型具有以標記來衡量的上下文視窗。上下文視窗定義您可以提供多少輸入以及模型可以產生多少輸出。您可以使用 API 或檢視模型文件來確定上下文視窗的大小。



In [ ]:
model_info = genai.get_model("models/gemini-1.5-flash")

# Returns the "context window" for the model,
# which is the combined input and output token limits.
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")
# ( input_token_limit=30720, output_token_limit=2048 )

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


## 計算符記數量
所有從 Gemini API 傳入和輸出的內容都會權杖化，包括文字、圖片 檔案以及其他非文字形式的內容

您可以透過下列方式計算符記：

 - 呼叫 count_tokens，並提供 。
這會傳回含有 可以先進行此呼叫，再將輸入內容傳送至模型 檢查要求大小

 - 呼叫後，在 response 物件上使用 usage_metadata 屬性 generate_content。
這會傳回輸入內容和輸出內容中的分詞總數： total_token_count。
也會分別傳回輸入和輸出內容的符記數量： prompt_token_count (輸入符記) 和 candidates_token_count (輸出符記)。



### 計算文字符記數量
如果透過純文字輸入內容呼叫 count_tokens，則會傳回符記數量 則用於輸入內容 (total_tokens)。你可以先撥打這通電話，時間早於 呼叫 generate_content 檢查要求大小。

另一種方式是呼叫 generate_content，然後使用 usage_metadata response 物件的屬性，就能取得下列資訊：

 - 輸入內容的個別符記數量 (prompt_token_count) 輸出內容 (candidates_token_count)
 - 輸入內容和輸出內容中的分詞總數 (total_token_count)

In [ ]:
model = genai.GenerativeModel("models/gemini-1.5-flash")

prompt = "敏捷的棕色狐狸跳過了那隻懶狗。"

# Call `count_tokens` to get the input token count (`total_tokens`).
print("total_tokens: ", model.count_tokens(prompt))
# ( total_tokens: 10 )

response = model.generate_content(prompt)

# On the response for `generate_content`, use `usage_metadata`
# to get separate input and output token counts
# (`prompt_token_count` and `candidates_token_count`, respectively),
# as well as the combined token count (`total_token_count`).
print(response.usage_metadata)
# ( prompt_token_count: 11, candidates_token_count: 73, total_token_count: 84 )

total_tokens:  total_tokens: 12

prompt_token_count: 13
candidates_token_count: 116
total_token_count: 129



### 計算多輪 (聊天) 符記數量
如果透過即時通訊記錄呼叫 count_tokens，會傳回總權杖 即時通訊中每個角色的文字數量 (total_tokens)。

另一種方式是呼叫 send_message，然後使用 usage_metadata response 物件的屬性，就能取得下列資訊：

 - 輸入內容的個別符記數量 (prompt_token_count) 輸出內容 (candidates_token_count)
 - 輸入內容和輸出內容中的分詞總數 (total_token_count)
如要瞭解下次對話回合的長度，您需要在 並在呼叫 count_tokens 時納入記錄。




In [ ]:
model = genai.GenerativeModel("models/gemini-1.5-flash")

chat = model.start_chat(
    history=[
        {"role": "user", "parts": "嗨，我的名字是凱文"},
        {"role": "model", "parts": "嗨，凱文!"},
    ]
)
# Call `count_tokens` to get the input token count (`total_tokens`).
print(model.count_tokens(chat.history))
# ( total_tokens: 10 )

response = chat.send_message(
    "用一句話向小朋友解釋計算機的工作原理。"
)

# On the response for `send_message`, use `usage_metadata`
# to get separate input and output token counts
# (`prompt_token_count` and `candidates_token_count`, respectively),
# as well as the combined token count (`total_token_count`).
print(response.usage_metadata)
# ( prompt_token_count: 25, candidates_token_count: 21, total_token_count: 46 )

from google.generativeai.types.content_types import to_contents

# You can call `count_tokens` on the combined history and content of the next turn.
print(model.count_tokens(chat.history + to_contents("生命的意義是什麼？")))
# ( total_tokens: 56 )

total_tokens: 14

prompt_token_count: 26
candidates_token_count: 34
total_token_count: 60

total_tokens: 67



## 計算多模態符記
提供給 Gemini API 的所有輸入內容都會權杖化，包括文字、圖片檔等 非文字形式的組合請注意以下關於權杖化的概略要點 的多模態輸入：

系統會將圖片視為固定大小，因此可耗用一定數量的圖片 符記 (目前 258 個符記)，不論其顯示或檔案大小。

影片和音訊檔案會按照下列固定費率轉換為權杖： 每秒有 263 個符記，音訊則每秒 32 個符記

圖片檔
在處理過程中，Gemini API 會將圖片視為固定大小， 使用固定數量的符記 (目前有 258 個符記) 螢幕或檔案大小

如果使用文字和圖片輸入內容呼叫 count_tokens，系統會傳回合併後的 文字和圖像的符記數量僅限輸入內容 (total_tokens)。個人中心 可以先進行此呼叫，再呼叫 generate_content 檢查 要求。您也可以選擇對文字和檔案呼叫 count_tokens 。

另一種方式是呼叫 generate_content，然後使用 usage_metadata response 物件的屬性，就能取得下列資訊：

 - 輸入內容的個別符記數量 (prompt_token_count) 輸出內容 (candidates_token_count)
 - 輸入內容和輸出內容中的分詞總數 (total_token_count)


In [ ]:
model = genai.GenerativeModel("models/gemini-1.5-flash")

prompt = "告訴我關於這張圖片的內容"
your_image_file = genai.upload_file(path="/content/moon_text.png")

# Call `count_tokens` to get the input token count
# of the combined text and file (`total_tokens`).
# An image's display or file size does not affect its token count.
# Optionally, you can call `count_tokens` for the text and file separately.
print(model.count_tokens([prompt, your_image_file]))
# ( total_tokens: 263 )

response = model.generate_content([prompt, your_image_file])
response.text
# On the response for `generate_content`, use `usage_metadata`
# to get separate input and output token counts
# (`prompt_token_count` and `candidates_token_count`, respectively),
# as well as the combined token count (`total_token_count`).
print(response.usage_metadata)
# ( prompt_token_count: 264, candidates_token_count: 80, total_token_count: 345 )

total_tokens: 266

prompt_token_count: 267
candidates_token_count: 79
total_token_count: 346



In [ ]:
import PIL.Image

model = genai.GenerativeModel("models/gemini-1.5-flash")

prompt = "告訴我關於這張圖片的內容"
your_image_file = PIL.Image.open("/content/moon_text.png")

# Call `count_tokens` to get the input token count
# of the combined text and file (`total_tokens`).
# An image's display or file size does not affect its token count.
# Optionally, you can call `count_tokens` for the text and file separately.
print(model.count_tokens([prompt, your_image_file]))
# ( total_tokens: 263 )

response = model.generate_content([prompt, your_image_file])

# On the response for `generate_content`, use `usage_metadata`
# to get separate input and output token counts
# (`prompt_token_count` and `candidates_token_count`, respectively),
# as well as the combined token count (`total_token_count`).
print(response.usage_metadata)
# ( prompt_token_count: 264, candidates_token_count: 80, total_token_count: 345 )

total_tokens: 266

prompt_token_count: 267
candidates_token_count: 43
total_token_count: 310



### 影片或音訊檔案
系統每個音訊和影片都會按照下列固定費率轉換為符記：

 - 影片：每秒 263 個符記
 - 音訊：每秒 32 個符記


如果使用文字和影片/音訊輸入內容呼叫 count_tokens，系統會傳回 僅限輸入內容中文字和影片/音訊檔案的綜合符記數量 (total_tokens).您可以先撥打這場通話，再撥號給 generate_content 檢查請求的大小您也可以選擇在以下位置呼叫 count_tokens： 可以分別遷移文字和檔案

另一種方式是呼叫 generate_content，然後使用 usage_metadata response 物件的屬性，就能取得下列資訊：

 - 輸入內容的個別符記數量 (prompt_token_count) 輸出內容 (candidates_token_count)
 - 輸入內容和輸出內容中的分詞總數 (total_token_count)


In [ ]:
!curl -O https://storage.googleapis.com/generativeai-downloads/data/Sherlock_Jr_FullMovie.mp4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  316M  100  316M    0     0  20.7M      0  0:00:15  0:00:15 --:--:-- 27.9M


In [ ]:
import time

model = genai.GenerativeModel("models/gemini-1.5-flash")

prompt = "告訴我關於這個影片的內容"
your_file = genai.upload_file(path="/content/Sherlock_Jr_FullMovie.mp4")

# Videos need to be processed before you can use them.
while your_file.state.name == "PROCESSING":
    print("processing video...")
    time.sleep(5)
    your_file = genai.get_file(your_file.name)

# Call `count_tokens` to get the input token count
# of the combined text and video/audio file (`total_tokens`).
# A video or audio file is converted to tokens at a fixed rate of tokens per second.
# Optionally, you can call `count_tokens` for the text and file separately.
print(model.count_tokens([prompt, your_file]))
# ( total_tokens: 300 )

response = model.generate_content([prompt, your_file])

# On the response for `generate_content`, use `usage_metadata`
# to get separate input and output token counts
# (`prompt_token_count` and `candidates_token_count`, respectively),
# as well as the combined token count (`total_token_count`).
print(response.usage_metadata)
# ( prompt_token_count: 301, candidates_token_count: 60, total_token_count: 361 )


processing video...
processing video...
processing video...
processing video...
processing video...
processing video...
processing video...
processing video...
total_tokens: 696168

prompt_token_count: 696169
candidates_token_count: 682
total_token_count: 696851



### 系統操作說明和工具
系統操作說明和工具也會計入 。

如果您採用系統指令，total_tokens 計數會增加以反映 增加 system_instruction。

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = "敏捷的棕色狐狸跳過了那隻懶狗。"

print(model.count_tokens(prompt))
# total_tokens: 10

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", system_instruction="你是一隻貓。你的名字是小灰。"
)

# The total token count includes everything sent to the `generate_content` request.
# When you use system instructions, the total token count increases.
print(model.count_tokens(prompt))
# ( total_tokens: 21 )

total_tokens: 12

total_tokens: 22



如果使用函式呼叫，total_tokens 計數會增加以反映 已新增 tools。

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = "我有 57 隻貓，每隻貓有 44 隻手套，總共有多少手套？"

print(model.count_tokens(prompt))
# ( total_tokens: 22 )

def add(a: float, b: float):
    """returns a + b."""
    return a + b

def subtract(a: float, b: float):
    """returns a - b."""
    return a - b

def multiply(a: float, b: float):
    """returns a * b."""
    return a * b

def divide(a: float, b: float):
    """returns a / b."""
    return a / b

model = genai.GenerativeModel(
    "models/gemini-1.5-flash-001", tools=[add, subtract, multiply, divide]
)

# The total token count includes everything sent to the `generate_content` request.
# When you use tools (like function calling), the total token count increases.
print(model.count_tokens(prompt))
# ( total_tokens: 206 )

total_tokens: 26

total_tokens: 210



In [ ]:
from google.colab import drive
drive.mount('/content/drive')